In [1]:
import numpy as np
import rasterio
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [2]:
def load_raster(file_path):
    with rasterio.open(file_path) as src:
        return src.read(), src.transform, src.crs

# Carga de imágenes RGB
rgb_images, transform, crs = load_raster('spatial_data/merged_raster_preprocessed.tiff')

# Carga de la máscara de clases
class_mask, _, _ = load_raster('spatial_data/badajoz_multiclass_mask.tif')

In [3]:
# Normalizar las imágenes RGB
rgb_images = rgb_images / 255.0

# Asegúrate de que los datos tienen las mismas dimensiones
assert rgb_images.shape[1:] == class_mask[0].shape, "Dimensiones x/y de las imágenes y máscaras no coinciden"

# Convertir las dimensiones de (bands, height, width) a (height, width, bands)
rgb_images = np.moveaxis(rgb_images, 0, -1)

# same with mask
class_mask = np.moveaxis(class_mask, 0, -1)

# check dimensions are correct
print(class_mask.shape)
print(rgb_images.shape)

(17281, 23085, 1)
(17281, 23085, 3)


In [ ]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(rgb_images, class_mask, test_size=0.2, random_state=42)

# Dividir los datos de entrenamiento en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [ ]:
def unet_model(input_size):
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoder
    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=-1)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=-1)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=-1)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = Conv2D(6, 1, activation='softmax')(conv9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Definir el tamaño de entrada basado en las dimensiones de tus imágenes
input_size = (rgb_images.shape[0], rgb_images.shape[1], rgb_images.shape[2])
model = unet_model(input_size)

In [ ]:
# train model
history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_val, y_val))

In [ ]:
# evaluate
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

In [ ]:
# predict and visualize
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=-1)